# DSCI 100: Introduction to Data Science

## Tutorial 2 - Getting data into R: Class activity

## Class Activity

1. Create a new R Jupyter notebook.

2. Connect to the `kickstarter` database (same database as question 3.0 from tutorial). Remember to load the packages necessary to do so.

3. Use the `projects` table from the `kickstarter` database to answer the following question: what is the project URL of the project from Great Britain (`GB`) with the greatest total amount pledged.

You **must** use all of the following functions once each: `arrange()`, `collect()`, `filter()`, `select()`, and `tail()`.

4. Post your code and final answer (project URL) to Piazza.

## Solution

In [1]:
library(tidyverse)
library(dbplyr)
library(DBI)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘dbplyr’

The following objects are masked from ‘package:dplyr’:

    ident, sql



In [2]:
connection <- dbConnect(RPostgres::Postgres(),
                       dbname = "kickstarter",
                       host = "r7k3-mds1.stat.ubc.ca",
                       port = 5432,
                       user = "dsci100",
                       password = "dsci100")

In [3]:
projects <- tbl(connection, "projects")
head(projects)

# Source:   lazy query [?? x 44]
# Database: postgres [dsci100@r7k3-mds1.stat.ubc.ca:5432/kickstarter]
      id state url_project url_project_sho… name  country creator_id location_id
   <int> <chr> <chr>       <chr>            <chr> <chr>        <int>       <int>
1 2.18e8 succ… https://ww… http://kck.st/1… Look… US      1436662259     2459115
2 2.18e8 succ… https://ww… http://kck.st/1… Iral… US      1954379699     2363557
3 2.18e8 succ… https://ww… http://kck.st/1… Kati… US       311393342     2495739
4 2.18e8 fail… https://ww… http://kck.st/d… Ange… US      1920930043     2391585
5 2.18e8 succ… https://ww… http://kck.st/1… 1001… US      1768556333     2410896
6 2.18e8 fail… https://ww… http://kck.st/2… LOON… US       692146147      565346
# … with 36 more variables: category_id <int>, created_at <int>,
#   deadline <int>, launched_at <int>, goal <dbl>, pledged <dbl>,
#   currency <chr>, currency_symbol <chr>, usd_pledged <dbl>,
#   static_usd_rate <dbl>, backers_count <int>, comments

In [4]:
colnames(projects)

[1] "id"                     "state"                  "url_project"           
 [4] "url_project_short"      "name"                   "country"               
 [7] "creator_id"             "location_id"            "category_id"           
[10] "created_at"             "deadline"               "launched_at"           
[13] "goal"                   "pledged"                "currency"              
[16] "currency_symbol"        "usd_pledged"            "static_usd_rate"       
[19] "backers_count"          "comments_count"         "updates_count"         
[22] "spotlight"              "staff_pick"             "blurb"                 
[25] "currency_trailing_code" "disable_communication"  "photo_url"             
[28] "profile_blurb"          "profile_id"             "profile_name"          
[31] "profile_project_id"     "profile_state"          "slug"                  
[34] "url_rewards"            "url_updates"            "video_id"              
[37] "video_url_high"         "video_url_webm"         "video_height"          
[40] "video_width"            "video_status"           "file_name"             
[43] "last_modification"      "deleted_comments"

In [5]:
projects_sel <- select(projects, url_project, country, pledged)
projects_filt <- filter(projects_sel, country == "GB")
projects_sorted <- arrange(projects_filt, pledged)
projects_coll <- collect(projects_sorted)
tail(projects_coll)

url_project,country,pledged
<chr>,<chr>,<dbl>
https://www.kickstarter.com/projects/awakenrealms/lords-of-hellas,GB,1717793
https://www.kickstarter.com/projects/playtonic/yooka-laylee-a-3d-platformer-rare-vival,GB,2090104
https://www.kickstarter.com/projects/torquing/zano-autonomous-intelligent-swarming-nano-drone,GB,2335120
https://www.kickstarter.com/projects/awakenrealms/nemesis-board-game,GB,3080834
https://www.kickstarter.com/projects/steamforged/dark-soulstm-the-board-game,GB,3771475
https://www.kickstarter.com/projects/awakenrealms/tainted-grail-the-fall-of-avalon,GB,4940031


## Question:

- Should we use `collect()` before or after working the data into our desired form? Does it matter?

- ANSWER: **Yes** it matters; we should use `collect()` **after**.

## But why?

- Databases are more efficient at selecting, filtering, and joining large datasets than R.

In [6]:
count(projects)

# Source:   lazy query [?? x 1]
# Database: postgres [dsci100@r7k3-mds1.stat.ubc.ca:5432/kickstarter]
  n      
  <int64>
1 408637 

- Thus, we should use `collect()` after working with our data in the database so our code is more efficient and we spend less time waiting for it to run.

Selecting and filtering the `projects` dataset (~408 thousand rows) in R:

In [7]:
ptm <- proc.time() ## Start a timer

projects_df <- collect(projects)
projects_df_selected <- select(projects_df, url_project, country, pledged)
projects_df_filtered <- filter(projects_df_selected,
                               country == "GB")
projects_df_sorted <- arrange(projects_df_filtered, pledged)
projects_df_tail <- tail(projects_df_sorted)

proc.time() - ptm ## Stop the timer

   user  system elapsed 
 11.848   0.660  15.758 

Doing the same, but using the database instead:

In [8]:
ptm <- proc.time() ## Start a timer

projects_selected <- select(projects, url_project, country, pledged)
projects_filtered <- filter(projects_selected,
                               country == "GB")
projects_sorted <- arrange(projects_filtered, pledged)
projects_collected <- collect(projects_sorted)
projects_tail <- tail(projects_collected)

proc.time() - ptm ## Stop the timer

   user  system elapsed 
  0.070   0.009   0.519 